## Objetivo: Classificar a chance de ocorrer um assalto de acordo com a localização, mês, dia do mês e dia da semana.

## 1- Importando a base de dados

In [ ]:
#  Importando as bibliotecas

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date

In [ ]:
url = 'https://raw.githubusercontent.com/robertferro/delitos_NY_2020/master/1%20-%20Manipula%C3%A7%C3%A3o%20de%20dados/dados_processados.csv'

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.head()

## 2- Preparaçao dos dados

In [ ]:
df.info()

 - Conventerndo a coluna data_da_ocorrencia para formato datetime

In [ ]:
df["data_da_ocorrencia"] = pd.to_datetime(df["data_da_ocorrencia"] )

In [ ]:
df.isna().sum()

In [ ]:
df.info()

 - Excluindo colunas que não serão utilizadas e renomeando o dataset.

In [ ]:
# como todos os registros correspondem ao ano de 2020, a coluna ano será exluída.
dados = df.drop(columns=['ano'])

In [ ]:
dados.shape

 ## Criando um filtro e pegando apenas os 10 delitos mais praticados

In [ ]:
dados.descricao_da_ofensa.value_counts().head(10)

In [ ]:
dados.descricao_da_ofensa.value_counts().head(10).sum()

In [ ]:
lista_delitos_recorrentes = dict(dados.descricao_da_ofensa.value_counts().head(15))
lista_delitos_recorrentes = list(lista_delitos_recorrentes.keys())

In [ ]:
dados = dados.set_index('descricao_da_ofensa')

In [ ]:
dados = dados.loc[lista_delitos_recorrentes]

In [ ]:
dados.assalto.value_counts()

In [ ]:
dados = dados.reset_index()

In [ ]:
dados.head()

In [ ]:
dados.shape

## 3- Visualização dos dados

In [ ]:
dados.describe()

In [ ]:
dados.corr().T

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(dados.corr(), annot=True, center=0, cmap="YlGnBu")
plt.show()

# 4 - Modelagem

## 4.1 Preparação dos Dados
- Separação Treino/Teste
- Separar Dados de Entrada (Atributos) e de saída (target)
- Tratar colunas categóricas
- Tratar dados ausentes


 - Convertendo variáveis categóricas em variáveis numéricas e definindo os dados de entrada e de saída.
 
      - No caso apenas as colunas que serão utilizadas na modelagem serão convertidas.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
enconder = LabelEncoder()

In [ ]:
dados.columns

In [ ]:
dados.info()

In [ ]:
# dados['nivel_da_ofensa'] = enconder.fit_transform(dados['nivel_da_ofensa'])
# dados['idade_do_infrator'] = enconder.fit_transform(dados['idade_do_infrator'])
# dados['descricao_da_ofensa'] = enconder.fit_transform(dados['descricao_da_ofensa'])
# dados['sexo_do_infrator'] = enconder.fit_transform(dados['sexo_do_infrator'])
# dados['raca_do_infrator'] = enconder.fit_transform(dados['raca_do_infrator'])
# dados['dia'] = enconder.fit_transform(dados['dia'])
# dados['mes'] = enconder.fit_transform(dados['mes'])
# dados['latitude'] = enconder.fit_transform(dados['latitude'])
# dados['longitude'] = enconder.fit_transform(dados['longitude'])
# dados['data_da_ocorrencia'] = enconder.fit_transform(dados['data_da_ocorrencia'])

In [ ]:
dados['bairro'] = enconder.fit_transform(dados['bairro'])
dados['dia_da_semana'] = enconder.fit_transform(dados['dia_da_semana'])

In [ ]:
dados.head()

Separando as variaveis em X e Y

In [ ]:
X = dados[['latitude', 'longitude','bairro','dia', 'mes', 'dia_da_semana']]
Y = dados['assalto']

 - Separando as amostras de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_treino,X_teste,Y_treino,Y_teste = train_test_split(X,Y,test_size=0.25,random_state=123 , stratify = Y)

 - Criando um Benchmark

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_treino, Y_treino)
y_pred = dummy_clf.predict(X_teste)
acuracia = dummy_clf.score(X_teste, Y_teste) * 100

print('A acurácia do dummy mostfrequent foi %.2f%%' % acuracia)

In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(Y_teste, y_pred)
sns.heatmap(cm, annot=True, fmt='g')

plt.xlabel('Predição( 1 ou 0)')
plt.ylabel('Real ( 1 ou 0)')

In [ ]:
from sklearn.metrics import roc_auc_score

y_proba = dummy_clf.predict_proba(X_teste)
roc_auc_score(Y_teste, y_proba[:, 1])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_teste, y_pred))

 - Fazendo uma triagem com outros modelos

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
lista_classificacores = [ LogisticRegression(),
                         DecisionTreeClassifier(),
                         RandomForestClassifier(),
                         XGBClassifier()]

for clf in lista_classificacores:
  print(clf.__class__.__name__)
  clf.fit(X_treino, Y_treino)

  print('Acuracia')
  train_acc = clf.score(X_treino, Y_treino)
  test_acc = clf.score(X_teste, Y_teste)
  print('Treino:', train_acc)
  print('Teste:', test_acc)

  print('ROC AUC')

  y_proba_treino = clf.predict_proba(X_treino)
  roc_treino = roc_auc_score(Y_treino, y_proba_treino[:, 1])
  print('Treino', roc_treino)

  y_proba_teste = clf.predict_proba(X_teste)
  roc_teste = roc_auc_score(Y_teste, y_proba_teste[:, 1])
  print('Test', roc_teste)
  print('='*80)

 - Escolhendo o modelo RandomForest, pois foi o que obteve o melhor score de teste e o melhor ROC AUC

In [ ]:
modelo = RandomForestClassifier()

In [ ]:
modelo.fit(X_treino, Y_treino)

In [ ]:
Y_previsto = modelo.predict(X_teste)

 - Gerando a matriz de confusão

In [ ]:
cm=confusion_matrix(Y_teste, Y_previsto)
sns.heatmap(cm, annot=True, fmt='g')

plt.xlabel('Predição( 1 ou 0)')
plt.ylabel('Real ( 1 ou 0)')

In [ ]:
from sklearn.metrics import roc_auc_score

y_proba = clf.predict_proba(X_teste)
roc_auc_score(Y_teste, y_proba[:, 1])

- Ajustando o treshold do modelo.

In [ ]:
cm=confusion_matrix(Y_teste, y_proba[:, 1] > 0.1)
sns.heatmap(cm, annot=True, fmt='g')

plt.xlabel('Predição( 1 ou 0)')
plt.ylabel('Real ( 1 ou 0)')

In [ ]:
y_proba = clf.predict_proba(X_teste)
roc_auc_score(Y_teste, y_proba[:, 1] >0.15)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_teste, y_proba[:, 1] > 0.15))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_teste, Y_previsto))

# GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parametros = {'n_estimators' : np.arange(80,210,10),
              'criterion' : ['gini','entropy'], 
              'max_depth' : np.arange(2,6),
              'min_samples_split' : np.arange(2,5),}
              'max_features' : ['auto','sqrt','log2']}


In [ ]:
# parametros = {'max_depth' : np.arange(1,10),
#               'criterion' : ['gini','entropy'],
#               }

In [ ]:
melhor_modelo = GridSearchCV(modelo, parametros, n_jobs=-1 ,  cv=5 , refit = True)

In [ ]:
melhor_modelo.fit(x_treino, y_treino)

In [ ]:
melhor_modelo.best_estimator_

In [ ]:
modelo_final = melhor_modelo.best_estimator_

In [ ]:
modelo_final.fit(X_treino, Y_treino)

In [ ]:
Y_previsto = modelo_final.predict(X_teste)

In [ ]:
score_teste = modelo_final.score(X_teste,Y_teste)
score_teste

In [ ]:
cm=confusion_matrix(Y_teste, Y_previsto)
sns.heatmap(cm, annot=True, fmt='g')

plt.xlabel('Predição( 1 ou 0)')
plt.ylabel('Real ( 1 ou 0)')

In [ ]:
y_proba = clf.predict_proba(x_teste)
roc_auc_score(Y_teste, y_proba[:, 1])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_teste, Y_previsto))